In [65]:
rdd = sc.textFile("../data/taxi-data-sorted-small.csv.bz2")
rdd = rdd.map(lambda x: x.split(','))

In [25]:
# import bz2
# with bz2.open("../data/taxi-data-sorted-small.csv.bz2", 'rt') as f:
#     content = f.readlines()

In [37]:
# rdd = sc.parallelize(content[:5000])

In [45]:
# rdd = rdd.map(lambda x: x.split(","))

In [66]:
import numpy as np

In [67]:
def isfloat(value):
    try:
        float(value)
        return True
    except:
        return False

def correctRows(p):
    if(len(p) == 17):
        if(isfloat(p[4]) and isfloat(p[5]) and isfloat(p[11]) and isfloat(p[12]) and isfloat(p[16])):
            if(float(p[4]) != 0 and float(p[5]) != 0 and float(p[11]) != 0 and float(p[16])):
                return p

In [68]:
texilinesCorrect = rdd.filter(correctRows)

In [69]:
n = texilinesCorrect.count()

In [96]:
data = texilinesCorrect.map(lambda x: (np.array([1.0, float(x[4]), \
                                         float(x[5]), float(x[11]), \
                                         float(x[12]), float(x[16])])))

In [133]:
# prev_w = prev_w = np.full(5, 0.1)
# WtX = data.map(lambda x: (np.multiply(prev_w, x[:-1]), x))\
#             .map(lambda x: (x[0].sum(), x[1]))

In [162]:
errors = WtX.map(lambda x: (np.subtract(x[0], x[1][5]), x[1]))

In [163]:
errors.take(1)

[(8.044, array([  1.  , 120.  ,   0.44,   3.5 ,   0.5 ,   4.5 ]))]

In [166]:
dW = errors.map(lambda x: (1, np.multiply(x[0], x[1][: -1])))\
        .reduceByKey(lambda x1, x2: np.add(x1, x2))

In [167]:
dW.collect()

[(1, array([1.10178961e+08, 1.13126232e+11, 5.24187237e+08, 1.91686238e+09,
         3.32014498e+07]))]

In [168]:
def gradientDescent(learningRate = 0.01, num_iteration = 100):
    prev_W= np.full(5, 0.1)
    count = 0
    precision = 0.00000001
    prev_stepsize = 1
    while(prev_stepsize > precision or num_iteration > count):
        
        WtX = data.map(lambda x: (np.multiply(prev_w, x[:-1]), x))\
            .map(lambda x: (x[0].sum(), x[1]))
        
        errors = WtX.map(lambda x: (np.subtract(x[0], x[1][5]), x[1]))
        
        dW_rdd = errors.map(lambda x: (1, np.multiply(x[0], x[1][:-1])))\
                    .reduceByKey(lambda x1, x2: np.add(x1, x2))\
                    .collect()
        
        dW = dW_rdd[0][1]
        
        
        cur_W = np.subtract(prev_W, learningRate*dW)
        
        print("Iteration: ", count)
        print("W: " ,cur_W)
        
        count += 1
        
        prev_stepsize = np.mean(np.abs(np.subtract(prev_W, cur_W)))
        
        prev_W = cur_W
        
    return cur_W

In [169]:
W = gradientDescent()

Iteration:  0
W:  [-1.10178951e+06 -1.13126232e+09 -5.24187227e+06 -1.91686237e+07
 -3.32014398e+05]
Iteration:  1
W:  [-2.20357912e+06 -2.26252463e+09 -1.04837446e+07 -3.83372475e+07
 -6.64028895e+05]
Iteration:  2
W:  [-3.30536873e+06 -3.39378695e+09 -1.57256170e+07 -5.75058713e+07
 -9.96043393e+05]
Iteration:  3
W:  [-4.40715834e+06 -4.52504927e+09 -2.09674894e+07 -7.66744951e+07
 -1.32805789e+06]
Iteration:  4
W:  [-5.50894795e+06 -5.65631159e+09 -2.62093617e+07 -9.58431190e+07
 -1.66007239e+06]
Iteration:  5
W:  [-6.61073756e+06 -6.78757390e+09 -3.14512341e+07 -1.15011743e+08
 -1.99208689e+06]
Iteration:  6
W:  [-7.71252717e+06 -7.91883622e+09 -3.66931065e+07 -1.34180367e+08
 -2.32410138e+06]
Iteration:  7
W:  [-8.81431678e+06 -9.05009854e+09 -4.19349788e+07 -1.53348990e+08
 -2.65611588e+06]


KeyboardInterrupt: 

In [51]:
# data = texilinesCorrect.flatMap(lambda x: [(1, np.array([1.0])), (4, np.array([float(x[4])])), \
#                                         (5, np.array([float(x[5])])), (11, np.array([float(x[11])])), \
#                                         (12, np.array([float(x[12])])), (16, np.array([float(x[16])]))])

In [53]:
# def appendToCol(x1, x2):
#     col_one = np.append(x1[0], x2[0])
#     col_idx4 = np.append(x1[1], x2[1])
#     col_idx5 = np.append(x1[2], x2[2])
#     col_idx11 = np.append(x1[3], x2[3])
#     col_idx12 = np.append(x1[4], x2[4])
#     col_idx16 = np.append(x1[5], x2[5])
#     return np.array([col_one, col_idx4, col_idx5, col_idx11, col_idx12, col_idx16])

In [63]:
# prev_w = np.full(5, 0.1)
# transposed = data.reduceByKey(lambda x1, x2: np.append(x1, x2))
# errors = transposed.map(lambda x: (x[0], np.multiply(0.1, x[1])))